In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir("../")


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [4]:
import os # Assuming you have a vars.py or similar file
exec(open("vars.py").read())
os.environ['MLFLOW_TRACKING_URI'] = MLFLOW_TRACKING_URI
os.environ['MLFLOW_TRACKING_USERNAME'] = MLFLOW_TRACKING_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = MLFLOW_TRACKING_PASSWORD


In [5]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

In [6]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        config = self.config.Evaluation
        eval_config = EvaluationConfig(
            path_of_model=config.path_of_model,
            training_data=config.training_data,
            mlflow_uri=config.mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [7]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            else:
                mlflow.keras.log_model(self.model, "model")

In [8]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
    
except Exception as e:
   raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 137s 271ms/step - loss: 2.9784 - accuracy: 0.8081


2023/08/17 13:47:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\rkuka\AppData\Local\Temp\tmp6z3i_3v6\model\data\model\assets


p:\DEEPCNNClassifier\env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/08/17 13:49:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 10
Created version '10' of model 'VGG16Model'.


In [9]:
import tensorflow as tf

# Check available GPUs
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

# Create a simple computation
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0], dtype=tf.float32)
b = tf.constant([5.0, 4.0, 3.0, 2.0, 1.0], dtype=tf.float32)
c = a + b

# TensorFlow will automatically use the GPU for the computation
print(c)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
tf.Tensor([6. 6. 6. 6. 6.], shape=(5,), dtype=float32)
